In [7]:
def predict_number(n):
    while n > 0:
        highest_power_of_2 = 1 << (n.bit_length() - 1)
        n = n - highest_power_of_2
        if n == 0:
            return 0
        elif n == highest_power_of_2 // 2:
            return 1
        elif n == highest_power_of_2 - 1:
            return 2

    return 0

# Reading input and processing
print(predict_number(5))

0


In [2]:
from google.cloud import videointelligence_v1 as videointelligence
from google.cloud import language_v1

# Function to transcribe video
def transcribe_video(gcs_uri):
    video_client = videointelligence.VideoIntelligenceServiceClient()
    features = [videointelligence.Feature.SPEECH_TRANSCRIPTION]
    operation = video_client.annotate_video(
        request={"features": features, "input_uri": gcs_uri}
    )
    result = operation.result(timeout=300)
    transcription = ""
    for speech_transcription in result.annotation_results[0].speech_transcriptions:
        transcription += speech_transcription.alternatives[0].transcript + " "
    return transcription.strip()

# Function to analyze transcription and extract steps
def analyze_text(text):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_syntax(document=document)
    
    steps = []
    current_step = ''
    for token in response.tokens:
        word = token.text.content
        if word.lower() in ['step', 'then', 'next', 'after']:
            if current_step:
                steps.append(current_step.strip())
            current_step = word
        else:
            current_step += ' ' + word

    if current_step:
        steps.append(current_step.strip())
    
    return steps

# Main function
def main():
    gcs_uri = 'gs://factory-work/VortexLens MVP Demo.mp4'
    transcription = transcribe_video(gcs_uri)
    steps = analyze_text(transcription)
    
    final_steps = []
    for idx, step in enumerate(steps):
        final_steps.append(f"Step {idx + 1}: {step}")

    for step in final_steps:
        print(step)

if __name__ == "__main__":
    main()


KeyboardInterrupt: 